In [1]:
import os
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [2]:
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import chromadb

In [3]:
def load_chunk_persist_pdf() -> Chroma:
    pdf_folder_path = "data/"
    documents = []
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PDFPlumberLoader(pdf_path)
            documents.extend(loader.load())
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=10)
    chunked_documents = text_splitter.split_documents(documents)
    client = chromadb.Client()
    if client.list_collections():
        consent_collection = client.create_collection("consent_collection")
    else:
        print("Collection already exists")
    vectordb = Chroma.from_documents(
        documents=chunked_documents,
        embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'),
        persist_directory="chroma_store/"
    )
    vectordb.persist()
    return vectordb

In [4]:
vectordb = load_chunk_persist_pdf()

Collection already exists


/Users/harheem/.local/share/virtualenvs/rag_tutorial-6Ll4mVWC/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
vectordb = Chroma(persist_directory="chroma_store/", embedding_function=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))
print(vectordb._collection.count())

23


In [15]:
matching_docs = vectordb.similarity_search(query)